In [30]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters, RDFlibConnection, BlazegraphConnection
import cimgraph.utils as utils

import importlib

In [31]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [32]:
params = ConnectionParameters(filename=None, cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

In [33]:
from cimbuilder.substation_builder import SingleBusSubstation

In [34]:
SubBuilder = SingleBusSubstation(connection=connection, name="single_sub", base_voltage=115000)
substation = SubBuilder.substation


In [35]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=connection, container=ieee13_feeder, distributed=False)

In [36]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=connection, container=assets13_feeder, distributed=False)

In [37]:
SubBuilder.new_single_bus_feeder(series_number= 10, feeder=ieee13_feeder, feeder_network=ieee13_network)
SubBuilder.new_single_bus_feeder(series_number = 20, feeder=assets13_feeder, feeder_network=assets13_network)

In [38]:
SubBuilder.network.pprint(cim.Substation)

{
    "4d01cb86-0940-4310-b563-acc5fc94cf89": {
        "mRID": "4d01cb86-0940-4310-b563-acc5fc94cf89",
        "name": "single_sub"
    }
}


In [39]:
SubBuilder.network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "NormalEnergizingSubstation": "4d01cb86-0940-4310-b563-acc5fc94cf89"
    },
    "5B816B93-7A5F-B64C-8460-47C17D6E4B0F": {
        "mRID": "5B816B93-7A5F-B64C-8460-47C17D6E4B0F",
        "NormalEnergizingSubstation": "4d01cb86-0940-4310-b563-acc5fc94cf89"
    }
}


In [40]:
utils.write_xml(SubBuilder.network, '../test_output/single_bus.xml')

In [41]:
# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)
blazegraph.execute('drop all')

/home/singha42/.cache/pypoetry/virtualenvs/cim-builder-hXthbBAa-py3.10/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=22ms, elapsed=22ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=33ms, commitTime=1708361486211, mutationCount=6461</p\n></html\n>'

In [42]:
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
loader = BlazegraphLoader(params)

In [43]:
loader.upload_from_file(filename='../test_models/IEEE13.xml')
loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
loader.upload_from_file(filename='../test_output/single_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="32"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="32"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="169" milliseconds="20"/>

In [44]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [45]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "4d01cb86-0940-4310-b563-acc5fc94cf89": {
        "mRID": "4d01cb86-0940-4310-b563-acc5fc94cf89",
        "name": "single_sub",
        "ConnectivityNodes": "['8a202d3d-ceba-44b6-9136-28877e0ad723', '96ba62d1-9c3b-4534-99d9-dba8a24d235f', 'a2ed531b-d3c1-4fa0-adf5-26384e95d2fe', 'ad5a2893-d25e-49a4-ac30-df79fcbb4c90', 'c32fac81-02bc-4cb0-b125-4c70ac8948f2']",
        "Equipments": "['89c21593-2639-444f-a01f-06703b4adeed', 'b826bdb2-4ee1-4cb7-a468-3baf60f3a897', 'c0986538-9262-4cef-9e0d-7b0ad6bb9863', '17a5fe81-9c43-43e3-b854-f693e0a61aab', '22dd107d-8a44-438a-bdb1-cb0468d7b213', '4934db02-6744-409f-9404-196abe62baff', '4b5fd476-3173-41fe-9317-1fa25bef79da']",
        "NormalEnergizedFeeder": "['49AD8E07-3BF9-A4E2-CB8F-C3722F837B62', '5B816B93-7A5F-B64C-8460-47C17D6E4B0F']"
    }
}


In [46]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['0F1E28C3-6C44-4F88-B79C-2FDBCA4487B2', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '421E99BE-A834-4809-B924-84D88F634A45', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '673E896A-DCBF-4E43-9924-BEB31C5B6005', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '76D6D03C-967C-4E12-836D-A98DF3552BC7', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '8C58660F-C62C-4903-BE72-22F1255B1E62', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', '94F822E0-7130-4205-8597-B47110BBEF4B', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', 'DBFA8EFA-B397-40DA-9642-60E8DE4B3471', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '0124E881-B82D-4206-BBDF-37D585159872', '04984C4D-CC

In [47]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


In [48]:
utils.get_all_data(network)

ConnectivityNode
Terminal
BusbarSection
Disconnector
Breaker
Substation
Feeder
TransformerTank
ACLineSegment
PowerTransformer
LoadBreakSwitch
EnergyConsumer
Fuse
PowerElectronicsConnection
EnergySource
LinearShuntCompensator
Recloser
Location
BaseVoltage
LoadResponseCharacteristic
EnergyConsumerPhase
OperationalLimitSet
TopologicalNode
TapChangerControl
TransformerTankEnd
PowerTransformerEnd
RegulatingControl
WireSpacingInfo
ACLineSegmentPhase
PerLengthPhaseImpedance
OverheadWireInfo
ConcentricNeutralCableInfo
TapeShieldCableInfo
PhaseImpedanceData
WirePosition
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
BatteryUnit
PowerElectronicsConnectionPhase
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
CoordinateSystem
PositionPoint
SubGeographicalRegion
SwitchPhase
LinearShuntCompensatorPhase
TopologicalIsland
GeographicalRegion


In [49]:
utils.write_xml(network, '../test_output/single_bus_and_feeders.xml')